<a href="https://colab.research.google.com/github/abhisinghh/PhishIntention_no_dynamic/blob/main/phishintention_edited.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/lindsey98/PhishIntention.git


fatal: destination path 'PhishIntention' already exists and is not an empty directory.


In [3]:
%cd PhishIntention


/content/PhishIntention


In [4]:
!git clone https://github.com/facebookresearch/detectron2.git

Cloning into 'detectron2'...
remote: Enumerating objects: 15280, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 15280 (delta 0), reused 1 (delta 0), pack-reused 15275
Receiving objects: 100% (15280/15280), 6.17 MiB | 22.34 MiB/s, done.
Resolving deltas: 100% (11111/11111), done.


In [5]:
!python -m pip install pyyaml==5.1
import sys, os, distutils.core
# Note: This is a faster way to install detectron2 in Colab, but it does not include all functionalities (e.g. compiled operators).
# See https://detectron2.readthedocs.io/tutorials/install.html for full installation instructions
!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

# Properly install detectron2. (Please do not install twice in both ways)
# !python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.2/274.2 kB 3.5 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
fatal: destination path 'detectron2' already exists and is not an empty directory.


No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 47.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 12.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61400 sha256=2b2b59486b41c48f03b9217847cf0c228da270dfbde7e6485d5d2e204f55a8eb
  Stored in directory: /root/.cache/pip/wheels/01/c0/af/77c1cf53a1be9e42a52b48e5af2169d40ec2e89f7362489dd0
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144555 sha256=d51ba2eaba8defbef5af5264b0daf82afebfa74d1dccad2fa3c6a7ffc103ed16
  Stored in directory: /root/.cache/pip/wheels/12/93/dd/1f6a

In [6]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0
torch:  2.1 ; cuda:  cu118
detectron2: 0.6


In [6]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [17]:
!pip install tldextract
!pip install helium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 36.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 904.6/904.6 kB 59.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 20.3 MB/s eta 0:00:00
  Created wheel for helium: filename=helium-3.2.6-py3-none-any.whl size=5961221 sha256=b439550998fee597d9540d1b8a5eb474fc5fae1abd515bc152b1bf76fd06864b
  Stored in directory: /root/.cache/pip/wheels/45/ab/fa/7274b8fc5d686105ac223a1d77a148f7f83255940dee4a81c0
Successfully built helium
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.7
    Uninstalling urllib3-2.0.7:
      Successfully uninstalled urllib3-2.0.7
  Attempting uninstall: selenium
    Found existing installation: selenium 4.15.2
    Uninstalling selenium-4.15.2:
      Successfully uninstalled selenium-4.15.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. Thi

In [8]:
import tldextract
import pickle
import numpy as np

In [9]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 548.5/548.5 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 56.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 448.3/448.3 kB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 68.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 68.8 MB/s eta 0:00:00


In [10]:
import gdown
!gdown --id 1zw2MViLSZRemrEsn2G-UzHRTPTfZpaEd

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1zw2MViLSZRemrEsn2G-UzHRTPTfZpaEd
To: /content/PhishIntention/src.zip
100% 1.27G/1.27G [00:17<00:00, 74.2MB/s]


In [13]:
!unzip /content/PhishIntention/src.zip

Archive:  /content/PhishIntention/src.zip
   creating: src/
   creating: src/OCR_siamese_utils/
  inflating: src/OCR_siamese_utils/.DS_Store  
  inflating: src/OCR_siamese_utils/config.py  
  inflating: src/OCR_siamese_utils/demo_downgrade.pth.tar  
   creating: src/OCR_siamese_utils/output/
  inflating: src/OCR_siamese_utils/output/.DS_Store  
   creating: src/OCR_siamese_utils/output/targetlist_lr0.01/
  inflating: src/OCR_siamese_utils/output/targetlist_lr0.01/bit.pth.tar  
   creating: src/OCR_siamese_utils/siamese_unified/
  inflating: src/OCR_siamese_utils/siamese_unified/.DS_Store  
  inflating: src/OCR_siamese_utils/siamese_unified/__init__.py  
   creating: src/OCR_siamese_utils/siamese_unified/bit_pytorch/
  inflating: src/OCR_siamese_utils/siamese_unified/bit_pytorch/.DS_Store  
  inflating: src/OCR_siamese_utils/siamese_unified/bit_pytorch/models.py  
  inflating: src/OCR_siamese_utils/siamese_unified/bit_pytorch/__init__.py  
  inflating: src/OCR_siamese_utils/siamese_unif

In [12]:
!rm -r '/content/PhishIntention/phishintention/src'


In [14]:
!mv '/content/PhishIntention/src' '/content/PhishIntention/phishintention'

In [15]:
import torch

In [20]:
from phishintention.phishintention_main import test
import matplotlib.pyplot as plt
from phishintention.phishintention_config import load_config
from phishintention.phishintention_main import element_recognition, phishpedia_classifier_OCR, credential_classifier_mixed_al, driver_loader, dynamic_analysis

# use full model
url = open("phishintention/datasets/test_sites/accounts.g.cdcde.com/info.txt").read().strip()
screenshot_path = "phishintention/datasets/test_sites/accounts.g.cdcde.com/shot.png"
device = 'cuda' if torch.cuda.is_available() else 'cpu'
cfg_path = '/content/PhishIntention/phishintention/configs_template.yaml' # None means use default config.yaml
AWL_MODEL, CRP_CLASSIFIER, CRP_LOCATOR_MODEL, SIAMESE_MODEL, OCR_MODEL, SIAMESE_THRE, LOGO_FEATS, LOGO_FILES, DOMAIN_MAP_PATH = load_config(cfg_path, device)

phish_category, pred_target, plotvis, siamese_conf, dynamic, _, pred_boxes, pred_classes = test(url, screenshot_path,
                                                                      AWL_MODEL, CRP_CLASSIFIER, CRP_LOCATOR_MODEL, SIAMESE_MODEL, OCR_MODEL, SIAMESE_THRE, LOGO_FEATS, LOGO_FILES, DOMAIN_MAP_PATH)

print('Phishing (1) or Benign (0) ?', phish_category)
print('What is its targeted brand if it is a phishing ?', pred_target)
print('What is the siamese matching confidence ?', siamese_conf)
print('Where are the predicted bounding boxes (in [x_min, y_min, x_max, y_max])?', pred_boxes)
plt.imshow(plotvis[:, :, ::-1])
plt.title("Predicted screenshot with annotations")
plt.show()

  pixel_mean
  pixel_std
  pixel_mean
  pixel_std


Load protected logo list


/usr/local/lib/python3.10/dist-packages/torch/__init__.py:614: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:451.)
  _C._set_default_tensor_type(t)
  0%|          | 0/277 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:4296: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(
100%|██████████| 277/277 [36:25<00:00,  7.89s/it]


Finish loading protected logo list
Entering phishintention


/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


plot
Entering siamese


/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:4296: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(


A target is reported by siamese, enter CRP classifier
Phishing is found!
Phishing (1) or Benign (0) ? 1
What is its targeted brand if it is a phishing ? Google
What is the siamese matching confidence ? 0.9679304
Where are the predicted bounding boxes (in [x_min, y_min, x_max, y_max])? tensor([[ 776.2476,  472.2879, 1141.6412,  526.9000],
        [ 732.8047,  293.6585, 1185.5934,  793.3476],
        [1061.0114,  676.0058, 1142.4022,  713.6852],
        [ 919.0002,  332.1234,  997.1187,  365.2018],
        [ 725.1760,  800.7636, 1231.9446,  841.1035],
        [ 772.9410,  683.0114,  882.6655,  703.5554],
        [1061.6277,  809.0794, 1133.9965,  835.2675],
        [ 964.5088,  802.3207, 1203.2163,  841.3578]])


In [27]:
import pickle
with open('/content/PhishIntention/phishintention/src/phishpedia_siamese/domain_map.pkl', 'rb') as f :
  domain_map  = pickle.load(f)

In [29]:
domain_map

{'Apple': ['apple.com', 'icloud.com'],
 'Credit Agricole S.A': ['credit-agricole.com'],
 'Key Bank': ['key.com'],
 'Webmail Provider': ['webmail-provider.com'],
 'Banco de Chile': ['bancochile.cl'],
 'Uber': ['uber.com'],
 'Comcast Corporation': ['comcast.com'],
 'BZSt': ['bzst.de'],
 'Navy Federal Credit Union': ['navyfederal.com'],
 'Bank of New Zealand': ['bnz.co.nz'],
 'CaixaBank': ['caixabank.com'],
 'Desjardins': ['desjardins.com'],
 'Nationwide Building Society': ['nationwide.com'],
 'SOCIETE FRANCAISE DU RADIOTELEPHONE': ['bloomberg.com'],
 'TradeKey': ['tradekey.com'],
 'Luno': ['luno.com'],
 'Discover': ['discover.com',
  'discovernetwork.com',
  'discoverglobalnetwork.com'],
 'Steam': ['steampowered.com', 'steamcommunity.com'],
 'ICICI Bank Limited': ['icicibank.com', 'icicigroupcompanies.com'],
 'Senior People Meet': ['seniorpeoplemeet.com'],
 'DocuSign': ['docusign.com'],
 'Bet365': ['bet365.com'],
 'amazon': ['amazon.com'],
 'ASB Bank Limited': ['asb.com'],
 'AutoScout24'